In [8]:
# !pip install numpy
# !pip install pandas
# !pip install tqdm
# !pip install scikit-learn
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error
from catboost import CatBoostRegressor

pd.set_option('display.max_columns', 200)

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
X_train = pd.read_csv("/content/drive/MyDrive/aviahack/X_train.csv")
y_train = pd.read_csv("/content/drive/MyDrive/aviahack/y_train.csv")

КОД ДЛЯ ОЧИСТКИ ДАТАСЕТА ОТ ПРОПУСКОВ

Используйте этот код если у вас нет уже очишенного от пропусков датасета

In [19]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

In [ ]:
## df = pd.merge(X_train, y_train, on=["engine_id", "flight_datetime", "flight_phase"])
# df = X_train
# df = df.drop(columns=[
#   "engine_id", "aircraft_id", "flight_datetime",
#   "flight_phase", "engine_family", "engine_type", "manufacturer",
#   "aircraft_family", "aircraft_type", "aircraft_grp", "ac_manufacturer"]) #Дропаем категориалььные признаки

# filler = KNNImputer()
# dfy = y_train
# dfy = dfy.drop(columns=["engine_id", "flight_phase", "flight_datetime"])

# dfy = filler.fit_transform(dfy)
# df = filler.fit_transform(df)

In [ ]:
# np.save('/content/drive/MyDrive/aviahack/dftrain',df)
# np.save('/content/drive/MyDrive/aviahack/dfytrain',dfy)

In [20]:
df = np.load('/content/drive/MyDrive/aviahack/dftrain.npy')
dfy = np.load('/content/drive/MyDrive/aviahack/dfytrain.npy') #Загрузка файла датасета без пропусков

In [27]:
scaler  = MinMaxScaler()
df = scaler.fit_transform(df)

In [37]:
dfs_cols = X_train.drop(columns=["engine_id", "aircraft_id", "flight_datetime",
                                  "flight_phase", "engine_family", "engine_type", "manufacturer",
                                  "aircraft_family", "aircraft_type", "aircraft_grp", "ac_manufacturer"]).columns
df = pd.DataFrame(df, columns=dfs_cols)
dfy_cols = y_train.drop(columns=["engine_id", "flight_phase", "flight_datetime"]).columns
dfy = pd.DataFrame(dfy, columns=dfy_cols)
df = df.drop(columns=['IHPSOV'])#Дропаем колонку, состоящую из нулей


In [35]:
def get_score_by_parameter(predict_field,df,dfy):
    
    X = df
    y = dfy[predict_field]

    
    Xm_train, Xm_test, ym_train, ym_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    model = CatBoostRegressor(iterations=100, verbose=False)
    model.fit(Xm_train, ym_train)
    catboost_predicted = model.predict(Xm_test)
    catboost_score = mean_squared_error(ym_test, catboost_predicted)
    return catboost_score

In [38]:
scores = pd.DataFrame(columns=["parameter", "rmse", ".0", ".1", ".5", ".9", "1"])
output_parameters = y_train.columns.drop(['flight_datetime', 'flight_phase', 'engine_id'])

for param in tqdm(output_parameters):
    try:
        score = get_score_by_parameter(param,df,dfy)
        qua = dfy[param].quantile([.0, .1, .5, .9, 1])
        qua = list(qua)
        scores.loc[len(scores)] = [param, score] + qua
    except Exception as error:
      print(error,'not good')
      continue



 73%|███████▎  | 22/30 [00:35<00:09,  1.23s/it]

catboost/libs/metrics/metric.cpp:6376: All train targets are equal not good


100%|██████████| 30/30 [00:49<00:00,  1.64s/it]


ВАЛИД

In [40]:
X_valid = pd.read_csv("/content/drive/MyDrive/aviahack/X_valid.csv")

КОД ДЛЯ ОЧИСТКИ ДАТАСЕТА ОТ ПРОПУСКОВ

Используйте этот код если у вас нет уже очишенного от пропусков датасета

In [41]:
# df_valid = X_valid
# df_valid = df_valid.drop(columns=[
#   "engine_id", "aircraft_id", "flight_datetime",
#   "flight_phase", "engine_family", "engine_type", "manufacturer",
#   "aircraft_family", "aircraft_type", "aircraft_grp", "ac_manufacturer",'IHPSOV'])

# filler = KNNImputer()

# df_valid = filler.fit_transform(df_valid)

KeyboardInterrupt: ignored

In [ ]:
# np.save('/content/drive/MyDrive/aviahack/dfvalid',df_valid)

In [42]:
df_valid = np.load('/content/drive/MyDrive/aviahack/dfvalid.npy')

In [43]:
scaler  = MinMaxScaler()
df_valid = scaler.fit_transform(df_valid)

In [44]:
df_valid_cols = X_valid.drop(columns=["engine_id", "aircraft_id", "flight_datetime",
                                  "flight_phase", "engine_family", "engine_type", "manufacturer",
                                  "aircraft_family", "aircraft_type", "aircraft_grp", "ac_manufacturer",'IHPSOV']).columns
df_valid = pd.DataFrame(df_valid, columns=df_valid_cols)

In [45]:
df_valid

,engine_position,n1_modifier,number_blades,ZHPTAC,ZLPTAC,ZPCN12,ZPCN25,ZPHSF,ZPHSR,ZPN12R,ZPOIL,ZPS3,ZT1AB,ZT3,ZT49,ZTAMB,ZTLA,ZTNAC,ZTOIL,ZVB1F,ZVB1R,ZVB2F,ZVB2R,ZVSV,ZWF36,AGW,CAS,IAI,IVS12,SAT,ZALT,ZT1A,ZVIAS,ZWBP1,ZWBP1_8E,ZWBP2,ZWBP2_8E,ZXM,IBE,IBP,IAIE
0,0.0,0.857143,1.000000,0.030108,0.020290,0.808766,0.733894,0.688889,0.044444,0.380429,0.134683,0.741618,0.440022,0.429848,0.697579,0.099173,0.783921,0.527397,0.057727,0.132300,0.233333,0.100,0.014222,0.126678,0.354183,0.072752,0.509740,0.0,0.0,0.461586,0.068436,0.539955,0.193222,0.008689,0.611268,0.008852,0.596319,0.244071,1.0,1.0,1.0
1,1.0,0.000000,0.333333,0.931183,0.866667,0.788977,0.652752,0.304444,0.517778,0.418902,0.087261,0.093391,0.317996,0.333498,0.730422,0.045390,0.243816,0.346575,0.119798,0.092507,0.120000,0.200,0.185556,0.144715,0.141103,0.069739,0.829870,0.0,0.0,0.363757,0.759134,0.457788,0.747846,0.007508,0.410867,0.007508,0.345603,0.702569,1.0,1.0,0.0
2,1.0,0.857143,1.000000,0.030108,0.020290,0.823313,0.753559,0.950000,0.094444,0.332224,0.130739,0.760631,0.377278,0.350124,0.719217,0.089786,0.787985,0.527397,0.053021,0.185220,0.066667,0.100,0.028444,0.136326,0.354779,0.072423,0.512987,0.0,0.0,0.461586,0.035162,0.632054,0.224698,0.000656,0.570671,0.000656,0.598957,0.239130,0.0,1.0,0.0
3,1.0,0.714286,1.000000,0.935484,0.898551,0.837859,0.747267,0.491667,0.225000,0.930503,0.054124,0.122270,0.288069,0.584017,0.681092,0.038093,0.650883,0.424658,0.115885,0.000000,0.100000,0.500,0.028444,0.191275,0.220677,0.075352,0.805195,0.0,0.0,0.186488,0.903620,0.483521,0.652613,0.006885,0.427547,0.006885,0.473456,0.759881,1.0,1.0,0.0
4,0.0,0.571429,0.000000,0.972043,0.921739,0.871606,0.736962,0.520000,0.403333,0.930503,0.063144,0.122270,0.288069,0.584017,0.694487,0.038093,0.650883,0.432877,0.128906,0.116424,0.266667,0.400,0.180222,0.094966,0.220760,0.716679,0.809740,0.2,0.0,0.429238,0.904193,0.459368,0.652613,0.007770,0.427547,0.007770,0.473456,0.771443,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47789,1.0,0.000000,0.333333,0.058065,0.026087,0.835646,0.706338,0.472222,0.348889,0.328516,0.103625,0.760536,0.854468,0.755059,0.876610,0.158779,0.798587,0.497260,0.098151,0.091473,0.156667,0.114,0.166667,0.155201,0.387085,0.018163,0.581623,0.0,0.0,0.564852,0.034825,0.670880,0.224009,0.004000,0.000000,0.003902,0.205930,0.257905,1.0,1.0,0.0
47790,1.0,0.285714,0.000000,0.030108,0.020290,0.899438,0.755133,0.221111,0.168889,0.343695,0.112248,0.740612,0.663813,0.605132,0.807831,0.132059,0.789222,0.527397,0.078080,0.164052,0.400000,0.020,0.019911,0.140101,0.367230,0.000341,0.617273,0.0,0.0,0.491446,0.054501,0.616253,0.180701,0.005279,0.219196,0.005475,0.222290,0.292984,1.0,1.0,0.0
47791,1.0,1.000000,1.000000,0.817204,0.811594,0.868891,0.762212,0.458333,0.294444,0.933150,0.055412,0.173945,0.277368,0.580218,0.718496,0.036380,0.679505,0.582192,0.115885,0.105840,0.000000,0.300,0.000000,0.171141,0.256380,0.084162,0.844156,0.0,0.0,0.410998,0.854695,0.517381,0.722229,0.008361,0.461975,0.008361,0.420961,0.758893,1.0,1.0,0.0
47792,1.0,0.000000,0.333333,0.058065,0.026087,0.828979,0.704372,0.278889,0.276667,0.319246,0.104606,0.763169,0.816300,0.735570,0.895286,0.151928,0.790635,0.497260,0.090621,0.210853,0.172667,0.088,0.339442,0.144715,0.372518,0.000380,0.579383,0.0,0.0,0.542457,0.033019,0.654628,0.282022,0.007377,0.000000,0.007246,0.000000,0.273715,1.0,1.0,0.0


In [46]:
def get_result(predict_field,df,X_valid):
    
    X = df
    # print(X.info())
    y = dfy[predict_field]
    # print(y)
    
    Xm_train, Xm_test, ym_train, ym_test = train_test_split(X, y, test_size=0.33, random_state=42)
    
    model = CatBoostRegressor(iterations=100, verbose=False)
    model.fit(Xm_train, ym_train)
    catboost_predicted = model.predict(Xm_test)
    catboost_score = mean_squared_error(ym_test, catboost_predicted)
    catboost_predicted = model.predict(X_valid)
    return catboost_predicted


In [47]:
output_parameters = y_train.columns.drop(['flight_datetime', 'flight_phase', 'engine_id'])
y_valid = pd.DataFrame(columns=[output_parameters],index=range(0,df_valid.shape[0]))

for param in tqdm(output_parameters):
    try:
        pred = get_result(param,df,df_valid)
        # print(pred)
        y_valid[param] = pd.Series(pred)
    except Exception as error:
      print(error,'not good')
      continue


 73%|███████▎  | 22/30 [00:36<00:09,  1.24s/it]

catboost/libs/metrics/metric.cpp:6376: All train targets are equal not good


100%|██████████| 30/30 [00:50<00:00,  1.69s/it]


In [ ]:
y_valid = y_valid.fillna(0)

In [ ]:
# y_valid.to_csv('/content/drive/MyDrive/aviahack/y_valid.csv') #https://drive.google.com/file/d/1-6nLV-cPCtO6_yhaGKmk2f9nGRBIal8I/view?usp=sharing

In [48]:
y_valid

,BRAT,DEGT,DELFN,DELN1,DELVSV,DPOIL,EGTC,EGTHDM,EGTHDM_D,GEGTMC,GN2MC,GPCN25,GWFM,PCN12,PCN12I,PCN1AR,PCN1BR,PCN1K,PCN2C,SLOATL,SLOATL_D,VSVNOM,WBE,WBI,WFMP,ZPCN25_D,ZT49_D,ZTLA_D,ZTNAC_D,ZWF36_D
0,0.931273,8.750982,19.187838,7.548760,10.319914,-16.410884,642.753710,74.494746,3.542334,116.273445,2.839524,0.109939,1.176385,82.735588,83.301788,84.897351,86.620826,85.144287,87.906944,65.965287,0.167254,NaN,1.152122,-0.037864,1819.618997,-0.039095,-5.735031,0.003516,-0.887896,54.941684
1,1.000651,18.141767,16.246642,6.657716,4.291596,-23.366273,690.520371,42.822697,1.079854,99.548610,3.737160,0.067188,2.968728,85.955437,85.996897,91.133381,80.753956,91.271974,87.065725,43.394558,0.747988,NaN,4.909538,-0.048163,1672.720383,-0.012455,-3.331860,-0.001688,-0.348237,0.891507
2,0.003099,-5.815755,22.011469,7.992863,7.222423,-19.944281,658.075702,87.096517,7.073008,118.501670,3.428223,-0.016198,-0.258285,83.988362,84.969506,83.608665,87.669563,83.618682,88.457838,72.572427,2.078876,NaN,0.539151,0.069772,2125.463367,-0.024782,-5.702497,0.010345,6.937706,19.806200
3,0.898188,-17.276408,17.483078,6.592017,1.910746,-23.168830,680.071843,62.258261,-2.246382,132.589482,2.969147,0.248962,-5.387088,86.402928,86.413044,88.709951,75.977410,91.151036,94.915475,64.699209,0.069008,NaN,0.642616,-0.109350,2832.545725,-0.012981,0.679247,0.022244,-8.314893,-6.197081
4,0.999884,23.686198,13.995558,5.501290,5.502155,-14.950828,651.016466,33.453784,-5.020230,56.557749,2.291455,-0.283717,2.484881,89.926045,90.161604,92.345659,87.553151,95.493317,93.868300,41.326243,-1.248512,NaN,1.507753,1.008070,2694.197677,-0.094072,5.076382,0.018901,-6.351869,1.656892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47789,1.000219,11.588162,17.867800,7.085645,2.543613,-14.745770,673.125417,40.313277,6.160688,60.893763,2.255637,-0.136530,1.295930,91.070721,91.187474,91.102848,68.782390,89.432069,90.063855,42.214685,2.066435,NaN,9.502314,0.167953,2239.600230,-0.014047,-4.875274,0.001829,-0.940880,-20.390120
47790,1.000550,10.459765,12.320243,5.515780,12.356980,-17.038482,709.973323,56.798205,-1.223242,60.338283,2.398807,-0.174804,-1.034745,92.432565,92.671205,92.202280,90.880352,92.309988,94.223372,46.196081,-0.352265,NaN,0.966463,-0.036313,2630.003721,-0.046232,1.592429,0.006291,0.506444,4.454871
47791,0.852161,-27.766071,14.249395,4.387131,0.785296,-23.956644,723.678305,39.126032,0.636302,84.030069,1.848182,-0.066635,-8.438586,89.621099,89.525622,92.201586,92.387285,93.015997,96.868022,44.366870,1.058042,NaN,1.159916,-0.127701,3436.507504,-0.060716,-3.510433,0.016549,0.338916,-33.116567
47792,1.001229,24.984498,18.855187,7.847985,2.577331,-15.986895,668.040211,17.108126,-12.983801,52.076177,2.077572,-0.075453,-0.294845,90.317336,90.418119,90.864378,88.331123,89.204453,90.379485,33.848944,-3.950811,NaN,2.939372,0.346885,2484.240079,0.117778,12.319920,-0.005450,-16.100519,-48.906882
